# Genetic Algorithms 
### Recipes Generator

Given an amount of calories, the model will provide several recipes that fit the amount given.

In [4]:
# Import Libraries 
import numpy as np
import random
import pandas as pd

int number = np.log(n) - 2

In [9]:
class Recipes_GAs:
    def __init__(self, calories, n_genes = 100, n_cromosomas = 3, split_point = None):
        """
            Genetic Algorithms that generate several recipes that fit with the amount of
            calories given.
            NOTE. The food is expected to be represented as 100gr per portion, as long as the 
                food is not contable, such as the rice, oatmeal, etc. On the other hand, can be
                represented per unit, for example, eggs, fruits, etc.
                
                ** PARAMETERS:
        """
        self.__calories = calories
        self.__n_genes = n_genes
        self.__n_cromosomas = n_cromosomas
        self.__split_point = split_point
        self.__food = pd.DataFrame({
            "food": ["Arroz Blanco", "Pechuga de Pollo", "Pan Integral", "Huevo", "Avena", 
                     "Soya", "Atún enlatado en aceite", "Salmon", "Brocoli", "Espinacas", "Apio",
                    "Helado", "Lentejas", "Yogur Natural", "Frijoles", "Nueces", "Almendras", 
                     "Remolacha", "Fresas", "Tortilla de Maiz"
                    ],
            "calories": [
                150, 175, 260, 70, 379, 150, 180, 206, 35, 25,
                18, 250, 132, 67, 130, 600, 579, 45, 35, 230
            ]
        })
    
    """
        Method to generate population, it will return the assigned food (food_type) and
        the amount for each type (food_amount).
        The 'minum' and 'maximum' values determine the range of random portions that can be chosen.
    """
    def _generate_population(self):
        minimum = 1
        maximum = int((self.__calories/np.mean(self.__food["calories"]))/self.__n_cromosomas)
        food_amount = np.random.randint(minimum, maximum, (self.__n_genes, self.__n_cromosomas))
        food_type = np.array([ [ self.__food[self.__food["food"] == random.choice(self.__food["food"])].index for j in range(self.__n_cromosomas)] for i in range(self.__n_genes) ]).reshape( (self.__n_genes, -1) )            
        return food_amount, food_type
        
    """
        This method calculate the total amount of calories based on the type of food and the 
        random number generated for it.
    """
    def _calculate_fitness(self, food_amount, food_type):
        food_calories = [ self.__food["calories"][ind] for ind in food_type ]
        fitness = [amount.dot(calories)  for amount, calories in zip(food_amount, food_calories)]
        return fitness
    
    """
        Roultte Selection, it will return two samples, moother and father.
        
    """
    def roulette_selection(self, fitness):
        total_fitness = np.sum(fitness)
        selected_sample = []
        for i in range(2):
            r = random.random()
            c = total_fitness * r
            Ca = 0
            for sample in fitness:
                Ca += sample
                if Ca >= c:
                    indx = fitness.index(sample)
                    selected_sample.append(indx)
                    break
        return selected_sample
    
    def generate_new_population(self, food_amount, food_type, fitness):
        new_population_amount = []
        new_population_type = []
        for i in range( int(self.__n_genes/2) ):
            selected_sample = self.roulette_selection(fitness)
            
            father_amount = food_amount[selected_sample[0]]
            father_type = food_type[selected_sample[0]]
            mother_amount = food_amount[selected_sample[1]]
            mother_type = food_type[selected_sample[1]]
            
            if(self.__split_point):
                split_p = self.__split_point
            else:
                split_p = random.randint(1, self.__n_genes - 2)
                
            child1_amount = np.concatenate([father_amount[:1], mother_amount[1:] ])
            child1_type = np.concatenate([father_type[:1], mother_type[1:]])
            child2_amount = np.concatenate([mother_amount[:1], father_amount[1:]])
            child2_type = np.concatenate([mother_type[:1], father_type[1:]])
            
            new_population_amount.append(child1_amount)
            new_population_amount.append(child2_amount)
            new_population_type.append(child1_type)
            new_population_type.append(child2_type)
            
        return new_population_amount, new_population_type
        
    
    def fit(self, epochs):
        food_amount, food_type = self._generate_population()
        
        for ite in range(epochs):
            fitness = self._calculate_fitness(food_amount, food_type)
            
            """
                If fitness equal to calories parameter.
            """
            if self.__calories in set(fitness):
                print(f"Iteracion: {ite}")
                ind = min(range(len(fitness)), key=lambda i: abs(fitness[i] - self.__calories))
                food_set = self.__food["food"][ food_type[ind] ]
                print(f"La receta es: \n Calorias {fitness[ind]} \n Receta: \n {food_set.values} \n Porciones: {food_amount[ind]}")
                break
                
            """ 
                If Fitness is bigger than calories parameter, breaks.
            """
            if max(fitness) > self.__calories:
                break
            
            new_population_amount, new_population_type = self.generate_new_population(food_amount, food_type, fitness)
            food_amount = np.array(new_population_amount)
            food_type = np.array(new_population_type)
            
        if self.__calories not in set(fitness):
            print(f"Iteracion: {ite}")
            ind = min(range(len(fitness)), key=lambda i: abs(fitness[i] - self.__calories))
            food_set = self.__food["food"][ food_type[ind] ]
            print(f"La receta mas cercana es: \n Calorias {fitness[ind]} \n Receta:\n {food_set.values} \n Porciones: {food_amount[ind]}")
            
        
    

In [10]:
obj = Recipes_GAs(calories= 8000, n_genes= 100, n_cromosomas= 5)
obj.fit(50)

Iteracion: 1
La receta mas cercana es: 
 Calorias 7993 
 Receta:
 ['Pechuga de Pollo' 'Brocoli' 'Nueces' 'Pechuga de Pollo' 'Almendras'] 
 Porciones: [7 4 4 1 7]
